In [186]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
import pytz
import json
import ta_formulas as ta
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import boto3
import ast

In [187]:
test_data = [{'contract_ticker': 'O:GE240705P00152500', 'quantity': 3, 'last_price': 0.5, 'volume': 262, 'spread_position': 0},
             {'contract_ticker': 'O:GE240705P00150000', 'quantity': 5, 'last_price': 0.23, 'volume': 158, 'spread_position': 1}]
test_contracts1 = test_data[0]['quantity']
test_ticker1 = 'GE'
start_date_str = '2024-07-01'
open_date = '2024-07-01 11:00'
key = "A_vXSwpuQ4hyNRj_8Rlw1WwVDWGgHbjp"
multiplier = 30
timespan = 'minute'

start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
weekday = start_date.weekday()
days_till_friday = (4 - weekday) % 7
end_date = start_date + timedelta(days = days_till_friday)
end_date_str = end_date.strftime('%Y-%m-%d')


In [188]:
def convert_timestamp_est(timestamp):
    # Create a UTC datetime object from the timestamp
    utc_datetime = datetime.fromtimestamp(timestamp).replace(tzinfo=pytz.utc)
    # Define the EST timezone
    est_timezone = pytz.timezone('America/New_York')
    # Convert the UTC datetime to EST
    est_datetime = utc_datetime.astimezone(est_timezone)
    return est_datetime

def polygon_call_stocks(contract, from_stamp, to_stamp, multiplier, timespan, open_date):
    trading_hours = ['09:30:00', '16:00:00']
    try:
        payload={}
        headers = {}
        url = f"https://api.polygon.io/v2/aggs/ticker/{contract}/range/{multiplier}/{timespan}/{from_stamp}/{to_stamp}?adjusted=true&sort=asc&limit=50000&apiKey={key}"
        response = requests.request("GET", url, headers=headers, data=payload)
        res_df = pd.DataFrame(json.loads(response.text)['results'])
        res_df['t'] = res_df['t'].apply(lambda x: int(x/1000))
        res_df['date'] = res_df['t'].apply(lambda x: convert_timestamp_est(x))
        res_df['year'] = res_df['date'].apply(lambda x: x.year)
        res_df['month'] = res_df['date'].apply(lambda x: x.month)
        res_df['time'] = res_df['date'].apply(lambda x: x.time())
        res_df['hour'] = res_df['date'].apply(lambda x: x.hour)
        res_df['minute'] = res_df['date'].apply(lambda x: x.minute)
        res_df['dt'] = res_df['date'].apply(lambda x: datetime(x.year, x.month, x.day,x.hour,x.minute))
        res_df = res_df[(res_df['time'] >= datetime.strptime(trading_hours[0], '%H:%M:%S').time()) & (res_df['time'] <= datetime.strptime(trading_hours[1], '%H:%M:%S').time())]
        res_df = res_df[res_df['dt'] >= datetime.strptime(open_date, '%Y-%m-%d %H:%M')]

        res_df.drop(['vw', 'h', 'l', 'c', 'n', 't','date', 'year','month', 'time', 'hour', 'minute'], axis = 1, inplace = True)
        res_df.reset_index(inplace=True)
        res_df.rename(columns = {'v': 'underlying_volume', 'o': 'underlying_price'}, inplace=True)
        res_df['symbol'] = contract
        res_df = res_df[['symbol', 'dt', 'underlying_price', 'underlying_volume']]

            
        return res_df
    except Exception as e:  
        print(e)
        return pd.DataFrame()

def polygon_call_options(contract, from_stamp, to_stamp, multiplier, timespan, open_date):
    trading_hours = ['09:30:00', '16:00:00']
    try:
        payload={}
        headers = {}
        url = f"https://api.polygon.io/v2/aggs/ticker/{contract}/range/{multiplier}/{timespan}/{from_stamp}/{to_stamp}?adjusted=true&sort=asc&limit=50000&apiKey={key}"
        response = requests.request("GET", url, headers=headers, data=payload)
        res_df = pd.DataFrame(json.loads(response.text)['results'])
        res_df['t'] = res_df['t'].apply(lambda x: int(x/1000))
        res_df['date'] = res_df['t'].apply(lambda x: convert_timestamp_est(x))
        res_df['year'] = res_df['date'].apply(lambda x: x.year)
        res_df['month'] = res_df['date'].apply(lambda x: x.month)
        res_df['time'] = res_df['date'].apply(lambda x: x.time())
        res_df['hour'] = res_df['date'].apply(lambda x: x.hour)
        res_df['minute'] = res_df['date'].apply(lambda x: x.minute)
        res_df['dt'] = res_df['date'].apply(lambda x: datetime(x.year, x.month, x.day,x.hour,x.minute))
        res_df = res_df[(res_df['time'] >= datetime.strptime(trading_hours[0], '%H:%M:%S').time()) & (res_df['time'] <= datetime.strptime(trading_hours[1], '%H:%M:%S').time())]
        res_df = res_df[res_df['dt'] >= datetime.strptime(open_date, '%Y-%m-%d %H:%M')]

        res_df.drop(['vw', 'h', 'l', 'c', 'n', 't','date', 'year','month', 'time', 'hour', 'minute'], axis = 1, inplace = True)
        res_df.reset_index(inplace=True)
        res_df.rename(columns = {'v': 'option_volume', 'o': 'option_price'}, inplace=True)
        res_df['option_contract'] = contract
        res_df = res_df[['option_contract', 'dt', 'option_price', 'option_volume']]
            
        return res_df
    except Exception as e:  
        print(e)
        return pd.DataFrame()

In [189]:
class CustomRetry(Retry):
    def is_retry(self, method, status_code, has_retry_after=False):
        """ Return True if we should retry the request, otherwise False. """
        if status_code != 200:
            return True
        return super().is_retry(method, status_code, has_retry_after)
    
def setup_session_retries(
    retries: int = 3,
    backoff_factor: float = 0.05,
    status_forcelist: tuple = (500, 502, 504),
):
    """
    Sets up a requests Session with retries.
    
    Parameters:
    - retries: Number of retries before giving up. Default is 3.
    - backoff_factor: A factor to use for exponential backoff. Default is 0.3.
    - status_forcelist: A tuple of HTTP status codes that should trigger a retry. Default is (500, 502, 504).

    Returns:
    - A requests Session object with retry configuration.
    """
    retry_strategy = CustomRetry(
        total=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
        allowed_methods=frozenset(["GET", "POST", "PUT", "DELETE", "HEAD", "OPTIONS"]),
        raise_on_status=False
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session = requests.Session()
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    
    return session

def execute_polygon_call(url):
    session = setup_session_retries()
    response = session.request("GET", url, headers={}, data={})
    return response 
    
def stat_window_creator(symbol, from_stamp, to_stamp, timespan, multiplier):
    trading_hours = [9,10,11,12,13,14,15]
    
    from_dt = datetime.strptime(from_stamp, '%Y-%m-%d')
    new_stamp = from_dt - timedelta(days = 1)
    from_stamp = new_stamp.strftime('%Y-%m-%d')

    data = []
    url = f"https://api.polygon.io/v2/aggs/ticker/{symbol}/range/{multiplier}/{timespan}/{from_stamp}/{to_stamp}?adjusted=true&sort=asc&limit=50000&apiKey={key}"
    response = execute_polygon_call(url)
    response_data = json.loads(response.text)
    results = response_data['results']
    results_df = pd.DataFrame(results)
    results_df['t'] = results_df['t'].apply(lambda x: int(x/1000))
    results_df['date'] = results_df['t'].apply(lambda x: convert_timestamp_est(x))
    results_df['hour'] = results_df['date'].apply(lambda x: x.hour)
    results_df['minute'] = results_df['date'].apply(lambda x: x.minute)
    results_df['day'] = results_df['date'].apply(lambda x: x.day)
    results_df['month'] = results_df['date'].apply(lambda x: x.month)
    results_df['symbol'] = symbol
    trimmed_df = results_df.loc[results_df['hour'].isin(trading_hours)]
    filtered_df = trimmed_df.loc[~((trimmed_df['hour'] == 9) & (trimmed_df['minute'] < 30))]

    return filtered_df



In [190]:
df = polygon_call_stocks(test_ticker1, start_date_str, end_date_str, multiplier, timespan, open_date)
df2 = polygon_call_options(test_ticker1, start_date_str, end_date_str, multiplier, timespan, open_date)
base_df = pd.merge(df, df2, on='dt')
base_df = base_df[['symbol', 'option_contract', 'dt', 'underlying_price', 'underlying_volume', 'option_price', 'option_volume']]
base_df.head(10)


,symbol,option_contract,dt,underlying_price,underlying_volume,option_price,option_volume
0,GE,GE,2024-07-01 11:00:00,159.3500,284722,159.3500,284722
1,GE,GE,2024-07-01 11:30:00,159.5500,601981,159.5500,601981
2,GE,GE,2024-07-01 12:00:00,158.9700,28483,158.9700,28483
3,GE,GE,2024-07-01 12:30:00,159.0100,13783,159.0100,13783
4,GE,GE,2024-07-01 13:00:00,159.4899,600,159.4899,600
5,GE,GE,2024-07-01 13:30:00,159.2200,437,159.2200,437
6,GE,GE,2024-07-01 15:00:00,159.0000,885,159.0000,885
7,GE,GE,2024-07-01 15:30:00,158.0284,490,158.0284,490
8,GE,GE,2024-07-02 09:30:00,161.4700,320112,161.4700,320112
9,GE,GE,2024-07-02 10:00:00,160.7402,112172,160.7402,112172


In [191]:
base_df['contracts'] = test_contracts1
base_df['contract_value'] = base_df['option_price'].apply(lambda x: (x * base_df['contracts'][0]).round(2))
window_df = stat_window_creator(base_df['symbol'][0], start_date_str, end_date_str, timespan, multiplier)
window_df['dt'] = window_df['date'].apply(lambda x: x.tz_localize(None))

window_df.reset_index(drop = True, inplace=True)
window_df.set_index('dt',inplace=True)

In [192]:
rsi = ta.rsi(window_df['c'], window=15)
roc = ta.roc(window_df['c'], window=15)

dataroc_df = pd.merge(base_df, roc, on = 'dt')
dataroc_df.rename(columns = {'c':'roc'}, inplace=True)
final_df = pd.merge(dataroc_df, rsi, on = 'dt')
final_df.rename(columns = {'c':'rsi'}, inplace=True)
final_df.head(100)

,symbol,option_contract,dt,underlying_price,underlying_volume,option_price,option_volume,contracts,contract_value,roc,rsi
0,GE,GE,2024-07-01 11:00:00,159.3500,284722,159.3500,284722,3,478.05,NaN,NaN
1,GE,GE,2024-07-01 11:30:00,159.5500,601981,159.5500,601981,3,478.65,NaN,NaN
2,GE,GE,2024-07-01 12:00:00,158.9700,28483,158.9700,28483,3,476.91,NaN,NaN
3,GE,GE,2024-07-01 12:30:00,159.0100,13783,159.0100,13783,3,477.03,NaN,NaN
4,GE,GE,2024-07-01 13:00:00,159.4899,600,159.4899,600,3,478.47,NaN,NaN
5,GE,GE,2024-07-01 13:30:00,159.2200,437,159.2200,437,3,477.66,NaN,NaN
6,GE,GE,2024-07-01 15:00:00,159.0000,885,159.0000,885,3,477.00,NaN,NaN
7,GE,GE,2024-07-01 15:30:00,158.0284,490,158.0284,490,3,474.09,NaN,NaN
8,GE,GE,2024-07-02 09:30:00,161.4700,320112,161.4700,320112,3,484.41,NaN,NaN
9,GE,GE,2024-07-02 10:00:00,160.7402,112172,160.7402,112172,3,482.22,NaN,NaN


In [193]:
# rsi_array = final_df['rsi'].to_numpy()
# roc_array = final_df['roc'].to_numpy()
# contract_value = final_df['contract_value'].to_numpy()
rsi_contract_cost_corr = final_df['rsi'].corr(final_df['contract_value'])
roc_contract_cost_corr = final_df['roc'].corr(final_df['contract_value'])
rsi_underlying_value_corr = final_df['rsi'].corr(final_df['underlying_price'])
roc_underlying_value_corr = final_df['roc'].corr(final_df['underlying_price'])
print(rsi_contract_cost_corr,roc_contract_cost_corr,rsi_underlying_value_corr,roc_underlying_value_corr)

0.7231776912381672 0.7532915684090257 0.7231923755841801 0.7531844976985695


In [194]:
# s3 = boto3.client('s3')
# key = f"invalerts_potential_trades/PROD_VAL/CDBFC_1D/2024/04/30/15.csv"
# contracts = s3.get_object(Bucket="inv-alerts-trading-data", Key=key)
# contracts = pd.read_csv(contracts['Body'])
# contract_df = pd.DataFrame(contracts).drop(columns= ['Unnamed: 0.1', 'Unnamed: 0'], axis = 1)
# contract_df['probabilities'] = contract_df['classifier_prediction'].astype(int)
# trade_data = contract_df[contract_df['classifier_prediction'] > 0.5]
# trade_data['trade_details1wk'] = trade_data['trade_details1wk'].apply(lambda x: ast.literal_eval(x))
# trade_data['num_contracts'] = trade_data['trade_details1wk'].apply(lambda x: len(x))
# trade_data = trade_data.loc[trade_data['num_contracts'] > 0]
# trade_data.head()
# final_df.rename(columns = {'c':'rsi'})